## Enriching stock market data using Open AI API 

<p align="center">
    <img src="images/nasdaq100.png" width="450">
</p>

The Nasdaq-100 is a stock market index made up of 101 equity securities issued by 100 of the largest non-financial companies listed on the Nasdaq stock exchange. It helps investors compare stock prices with previous prices to determine market performance.

In this project you are provided with two CSV files containing Nasdaq-100 stock information:
- _**nasdaq100_CA.csv**_: contains information about companies in the index such as symbol, name, etc. For this analysis, only companies headquartered in California have been selected.
- _**nasdaq100_price_change.csv**_: contains price changes per stock across periods including (but not limited to) one day, five days, one month, six months, one year, etc.

As an AI developer, you will leverage the OpenAI API to classify companies into sectors and produce a summary of sector and company performance for this year, for the companies in the index that are headquartered in California.

# CSV with Nasdaq-100 stock data

In this project, you have available two CSV files `nasdaq100_CA.csv` and `nasdaq100_price_change.csv`.

## nasdaq100_CA.csv

```py
symbol,name,headQuarter,dateFirstAdded,cik,founded
AAPL,Apple Inc.,"Cupertino, CA",,0000320193,1976-04-01
ABNB,Airbnb,"San Francisco, CA",,0001559720,2008-08-01
ADBE,Adobe Inc.,"San Jose, CA",,0000796343,1982-12-01
...
```

## nasdaq100_price_change.csv

```py
symbol,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
AAPL,-1.7254,-8.30086,-6.20411,3.042,15.64824,42.99992,8.47941,60.96299,245.42031,976.99441,139245.53954
ABNB,2.1617,-2.21919,9.88336,19.43286,19.64241,68.66902,23.64013,-1.04347,-1.04347,-1.04347,-1.04347
ADBE,0.5409,-1.77817,9.16191,52.0465,38.01522,57.22723,21.96206,17.83037,109.05718,1024.69214,251030.66399
ADI,0.9291,-4.03352,2.58486,3.65887,5.01602,17.02062,8.09735,63.42847,92.81874,286.77518,26012.63736
...
1. ```

In [63]:
# Start your code here!
import os
import pandas as pd
from openai import OpenAI

# Instantiate an API client
client = OpenAI()

nasdaq100_ca = pd.read_csv("nasdaq100_CA.csv")
nasdaq = pd.read_csv("nasdaq100_price_change.csv")
# Continue coding here
# Use as many cells as you like

In [64]:
nasdaq100_ca = nasdaq100_ca.merge(nasdaq[["symbol", "ytd"]], on="symbol", how="left")
sectors = {
    "Technology",
    "Consumer Cyclical",
    "Industrials",
    "Utilities",
    "Healthcare",
    "Communication",
    "Energy",
    "Misc",
}

def sortbysector(symbol: str) -> str:
    prompt = (
        "You are trying to sort these companies by their company sector"
        "Classify the PRIMARY sector for the public company with stock ticker "
        f"'{symbol}' into ONE from the following:\n"
        f"{', '.join(sectors)}.\n\n"
        "Return exactly one of these sector names"
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You classify companies into stock market     sectors."},
            {"role": "user", "content": prompt},
        ],
        temperature = 0.0,
    )

    category = response.choices[0].message.content.strip()
    if category not in sectors:
        category = "Misc"

    return category


In [65]:


def sortbysector(symbol):
    # Example sector mapping (replace with your actual logic)
    sectors = {
        "AAPL": "Technology",
        "MSFT": "Technology",
        "GOOG": "Technology",
        "AMZN": "Consumer Discretionary",
        "TSLA": "Consumer Discretionary",
        # ... add more mappings as needed
    }
    # Use .get() with a default value to avoid UnboundLocalError
    return sectors.get(symbol, "Misc")

# Now the rest of your code
nasdaq["sector"] = nasdaq["symbol"].apply(sortbysector)
nasdaq100_ca = nasdaq100_ca.merge(
    nasdaq[["symbol", "sector"]],
    on="symbol",
    how="left"
)
summary_table = nasdaq[["symbol", "ytd", "sector"]].to_csv(index=False)

summary_prompt = (
    "You are an expert stock analyst. Given this table of selected Nasdaq companies "
    "with the following columns: symbol, ytd, and sector. "
    "Table:\n"
    f"{summary_table}\n\n"
    "Summarize the performance of each sector, choose the 2 best sectors (look at YTD) "
    "and evaluate the risk to reward. Explain your rationale concisely."
)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You write concise, clear investment summaries (not financial advice)."},
        {"role": "user", "content": summary_prompt},
    ],
    temperature=0.3,
)

stock_recommendations = response.choices[0].message.content.strip()
print(stock_recommendations)


### Sector Performance Summary

1. **Technology**
   - Notable Companies: AAPL, GOOG, MSFT, NVDA
   - Average YTD Performance: Approximately 59.8%
   - Key Highlights: NVDA leads with a staggering 217.27% YTD return, while AAPL and MSFT also show strong gains. The sector benefits from ongoing advancements in AI and cloud computing.

2. **Consumer Discretionary**
   - Notable Companies: AMZN, TSLA
   - Average YTD Performance: Approximately 66.2%
   - Key Highlights: TSLA shows a remarkable 132.61% YTD return, and AMZN contributes positively as well. The sector is buoyed by strong consumer spending and recovery post-pandemic.

3. **Miscellaneous**
   - Average YTD Performance: Approximately 20.5%
   - Key Highlights: This sector includes a mix of companies with varying performances. While some companies like META and MRVL show significant gains, others are underperforming.

### Best Sectors

1. **Consumer Discretionary**
   - **Rationale**: Strong performance driven by leading companies